# Charts for bus service opportunities by tract and CalEnviroScreen / Density Scores

* Density and CalEnviroScreen both grouped as low/med/high
* Provide some charts to accompany maps
* Heatmaps to show distribution?
* Scatterplot of pop density vs service density

In [1]:
import pandas as pd

import make_tract_viz

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
CHART_IMG_PATH = "img/"

df = make_tract_viz.import_processed_data()
#make_tract_viz.create_charts(df, CHART_IMG_PATH)
#make_tract_viz.create_maps(df, CHART_IMG_PATH)

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract


In [3]:
TOTAL_CA_TRACTS = len(df)

![](./img/arrivals_per_1k_popjobdensity.png)
![](./img/arrivals_per_1k_equity.png)
![](./img/arrivals_per_1k_bar.png)

## Opportunity Tracts

Tracts in the lower right quandrant, but pick out the most extreme cases.

Tracts with > 75th ptile for pop/job density & < 25th ptile for service density.

Especially the medium-need tracts, these would be the ones dragging down the average for the middle yellow bar.

In [4]:
# Grab the ones with density above 75th percentile
# and in the lower 25th percentile for service density
p75 = df.popjobs_sq_mi.quantile(0.75)
p25 = df.arrivals_per_1k_pj.quantile(0.25)

df = df.assign(
    flag_me = df.apply(lambda x: (x.popjobs_sq_mi > p75) and 
                       (x.arrivals_per_1k_pj < p25), axis=1)
)

pct_oppor_tracts = round(len(df[df.flag_me==1])/TOTAL_CA_TRACTS, 3)
print(f"# opportunity tracts: {len(df[df.flag_me==1])}")
print(f"% opportunity tracts of total: {pct_oppor_tracts*100}%")

# opportunity tracts: 167
% opportunity tracts of total: 2.1%


### List these cities where opportunity tracts fall

In [5]:
display_cols = [
    "County", "City", "ZIP"
]

for county in ["Santa Clara", "Los Angeles", "Sacramento"]: 
    df2 = df[df.flag_me==1].reset_index(drop=True)
    df3 = df2.drop_duplicates(subset=["County", "City"])
    display(df3[df3.County==county][display_cols])
    print(df3[df3.County==county].City.unique())

,County,City,ZIP
69,Santa Clara,San Jose,95112
89,Santa Clara,Campbell,95008
98,Santa Clara,Unincorporated Santa Clara County area,95111
118,Santa Clara,Alum Rock,95127
124,Santa Clara,Milpitas,95035
125,Santa Clara,Sunnyvale,94089
128,Santa Clara,Santa Clara,95054
143,Santa Clara,Cupertino,95014
151,Santa Clara,Mountain View,94041
156,Santa Clara,Palo Alto,94306


['San Jose' 'Campbell' 'Unincorporated Santa Clara County area'
 'Alum Rock' 'Milpitas' 'Sunnyvale' 'Santa Clara' 'Cupertino'
 'Mountain View' 'Palo Alto']


,County,City,ZIP
5,Los Angeles,Los Angeles,91352
11,Los Angeles,Glendale,91207
12,Los Angeles,Burbank,91501
14,Los Angeles,El Monte,91732
15,Los Angeles,Rosemead,91770
16,Los Angeles,Whittier,90601
18,Los Angeles,La Mirada,90638
19,Los Angeles,Montebello,90640
23,Los Angeles,Maywood,90270
24,Los Angeles,Huntington Park,90255


['Los Angeles' 'Glendale' 'Burbank' 'El Monte' 'Rosemead' 'Whittier'
 'La Mirada' 'Montebello' 'Maywood' 'Huntington Park' 'Cudahy'
 'South Gate' 'Bellflower' 'Culver City']


,County,City,ZIP
38,Sacramento,Sacramento,95816
51,Sacramento,Arden-Arcade,95825
56,Sacramento,Foothill Farms,95842
58,Sacramento,Folsom,95630
59,Sacramento,Rancho Cordova,95670
60,Sacramento,Rosemont,95826


['Sacramento' 'Arden-Arcade' 'Foothill Farms' 'Folsom' 'Rancho Cordova'
 'Rosemont']
